# 1. Instalacion Spark y el entorno

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xvf spark-3.5.1-bin-hadoop3.tgz

In [2]:
!ls

sample_data  spark-3.5.1-bin-hadoop3  spark-3.5.1-bin-hadoop3.tgz


In [3]:
!pip install -q findspark

In [4]:
!ls /usr/lib/jvm

java-1.11.0-openjdk-amd64  java-11-openjdk-amd64  java-1.8.0-openjdk-amd64  java-8-openjdk-amd64


In [5]:
!ls /usr/lib/jvm/java-8-openjdk-amd64/

ASSEMBLY_EXCEPTION  bin  docs  include	jre  lib  man  src.zip	THIRD_PARTY_README


In [8]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_402"
OpenJDK Runtime Environment (build 1.8.0_402-8u402-ga-2ubuntu1~22.04-b06)
OpenJDK 64-Bit Server VM (build 25.402-b06, mixed mode)


In [9]:
import findspark
findspark.init()

In [12]:
from posix import RWF_NOWAIT
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.sql.types import Row
from pyspark.sql import SQLContext

In [13]:
sc = SparkContext(master="local", appName="DataFrames")
sqlContext = SQLContext(sc)

/content/spark-3.5.1-bin-hadoop3/python/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [15]:
#Crear desde RDD un dataframe
deportistaOlimpicoRDD = sc.textFile("deportista.csv").map(lambda linea: linea.split(","))

In [16]:
deportistaOlimpicoRDD.take(5)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

In [19]:
#Crear función para eliminar encabezados
def eliminaEncabezado(index,iterator):
  return iter(list(iterator)[1:])

In [20]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.mapPartitionsWithIndex(eliminaEncabezado)

In [21]:
deportistaOlimpicoRDD.take(5)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278'],
 ['5', 'Christine Jacoba Aaftink', '2', '21', '185', '82', '705']]

Ahora se tienen que definir qué tipos de datos son los datos, ya sean enteros o string

In [22]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.map(lambda e: (
    int(e[0]),
    e[1],
    int(e[2]),
    int(e[3]),
    int(s[4]),
    float(e[5]),
    int(e[6])
    ))

Ahora vamos a declarar una variable que se va a llamar esquema, para ello se usa la clase objeto StructType ue recbie un arreglo done cada posición son las columns que se esperan

['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],

In [40]:
schema = StructType([
 StructField("deportista_id",IntegerType(),False),
 StructField("nombre",StringType(),False),
 StructField("genero",IntegerType(),False),
 StructField("edad", IntegerType(),False),
 StructField("altura",IntegerType(),False),
 StructField("peso",FloatType(),False),
 StructField("equipo_id",IntegerType(),False)
])

Juegos: ['', 'nombre_juego', 'annio', 'temporada', 'ciudad']

In [44]:
#crear dataframe desde archivo
juegoSchema = StructType([
    StructField("juego_id",IntegerType(),False),
    StructField("nombre_juego",StringType(),False),
    StructField("annio",IntegerType(),False),
    StructField("temporada",StringType(),False),
    StructField("ciudad",StringType(),False),
])

In [46]:
juegosDF = sqlContext.read.schema(juegoSchema).option("header","true").csv("juegos.csv")

**FLANTAN COLUMNAS**

In [47]:
from pyspark.sql.functions import *

In [49]:
deportistaDFSQL = deportistaDF.select("deportista_id", "nombre", col("edad").alias("edadAlJugar"), "equipo_id")

NameError: name 'deportistaDF' is not defined